In [4]:
import time
import numpy as np
import pandas as pd
import time
import pickle
import sys
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [5]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https:/1xbet.whoscored.com/')
driver.implicitly_wait(3)
driver.close()

In [6]:
def crawling_player_statistics(URL):
    """
    crawling player statistics of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(URL)    
    
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()
    driver.find_element(By.XPATH, '//*[@id="apps"]/dd[2]/a').click()
    time.sleep(5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-summary > div > dl.listbox.right > dt > b')[0].get_attribute("textContent").split("\\")[0]
    page = int(page.split('/')[1].split(' ')[0])
    player_statistics_df = pd.DataFrame(columns=[
        "player_name", "team_number", "team_name", "Apps", "Mins", "Goals",
        "Assists", "Yel", "Red", "SpG","PS%", "AerialsWon","MoM","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
      
        for i in range(page):
            if i != 0:
                driver.find_element(By.ID, 'next').click()
                time.sleep(1.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "Apps", "Mins", "Goals",
                "Assists", "Yel", "Red", "SpG","PS%", "AerialsWon","MoM","Rating"])
            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, ("a"))[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Goals": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Yel": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Red": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "SpG": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "PS%": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "AerialsWon": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "MoM": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict

            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
            
    driver.close()
    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)
    

In [7]:
def crawling_player_statistics_defensive(URL):
    """
    crawling player statistics(defensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)   

    time.sleep(1.5) 
    
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()

    time.sleep(1.5)
    
    driver.find_element(By.LINK_TEXT, "Defensive").click()

    
    time.sleep(1.5)

    
    driver.find_element(By.XPATH, '/html/body/div[5]/div[5]/div[3]/div[1]/div[2]/dl[2]/dd[2]/a').click()


    time.sleep(2.5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-defensive > div > dl.listbox.right > dt > b')[0].get_attribute("textContent")
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Tackles",
            "Inter", "Fouls", "Offsides", "Clear","Drb", "Blocks","OwnG","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(1.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Tackles",
                "Inter", "Fouls", "Offsides", "Clear","Drb", "Blocks","OwnG","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Tackles": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Inter": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Fouls": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Offsides": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Clear": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Drb": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Blocks": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "OwnG": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    
    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)
    

In [8]:
def crawling_player_statistics_offensive(URL):
    """
    crawling player statistics(offensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)   

    time.sleep(1.5) 
    
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()

    time.sleep(1.5)
    
    driver.find_element(By.LINK_TEXT, "Offensive").click()

    
    time.sleep(1.5)

    
    driver.find_element(By.LINK_TEXT, "All players").click()
    
    time.sleep(1.5)

    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-offensive > div > dl.listbox.right > dt > b')[0].get_attribute("textContent").split("\t")[0]
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Goals",
            "Assists", "SpG", "KeyP", "Dribble","Fouled", "Off","Disp","UnsTch","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(1.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Goals",
                "Assists", "SpG", "KeyP", "Dribble","Fouled", "Off","Disp","UnsTch","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR,"a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR,"span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Goals": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "SpG": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "KeyP": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Dribble": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Fouled": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Off": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Disp": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "UnsTch": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[13].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()

    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)

In [9]:
def crawling_player_statistics_passing(URL):

    url = str(URL)
    options = Options()
    options.add_argument("start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)   

    time.sleep(1.5) 
    
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()

    time.sleep(1.5)
    
    driver.find_element(By.LINK_TEXT, "Passing").click()

    
    time.sleep(1.5)

    
    driver.find_element(By.LINK_TEXT, "All players").click()


    time.sleep(1.5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-passing > div > dl.listbox.right > dt > b')[0].get_attribute("textContent")
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Assists",
            "KeyP", "AvgP","PS%", "Crosses","LongB","ThrB","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(1.5)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins","Assists",
                "KeyP", "AvgP","PS%", "Crosses","LongB","ThrB","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "KeyP": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "AvgP": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "PS%": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Crosses": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "LongB": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "ThrB": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    player_statistics_df.replace("-", "0", inplace=True)

    return(player_statistics_df)

# crwaling player stats at once

In [10]:
def crwaling_player_stats_at_once(URL):
    stats_summary = crawling_player_statistics(URL)
    print('player stats summary : done')
    stats_defensive = crawling_player_statistics_defensive(URL)
    print('player stats defensive : done')
    stats_offensive = crawling_player_statistics_offensive(URL)
    print('player stats offensive : done')
    stats_passing = crawling_player_statistics_passing(URL)
    print('player stats pasing : done')
    player_stats = pd.merge(pd.merge(stats_passing, stats_summary), 
                            pd.merge(stats_offensive, stats_defensive))
    return(player_stats)

In [11]:
SerieA = [
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/PlayerStatistics/Italy-Serie-A-2016-2017',
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Stages/15404/PlayerStatistics/Italy-Serie-A-2017-2018',
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/7468/Stages/16548/PlayerStatistics/Italy-Serie-A-2018-2019',
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/7928/Stages/17835/PlayerStatistics/Italy-Serie-A-2019-2020',
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/8330/Stages/18873/PlayerStatistics/Italy-Serie-A-2020-2021',
'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/8735/Stages/19982/PlayerStatistics/Italy-Serie-A-2021-2022'
]

Laliga = [
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/6436/Stages/13955/PlayerStatistics/Spain-LaLiga-2016-2017',
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/6960/Stages/15375/PlayerStatistics/Spain-LaLiga-2017-2018',
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/7466/Stages/16546/PlayerStatistics/Spain-LaLiga-2018-2019',
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/7889/Stages/17702/PlayerStatistics/Spain-LaLiga-2019-2020',
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/8321/Stages/18851/PlayerStatistics/Spain-LaLiga-2020-2021',
'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/8681/Stages/19895/PlayerStatistics/Spain-LaLiga-2021-2022'
]

for i in Premier_league:
  Data = crwaling_player_stats_at_once(i)
  Data.to_csv(i.split('/')[12]+'.csv')

page 59: 100%|██████████| 59/59 [07:25<00:00,  7.55s/it]
player stats summary : done


WebDriverException: Message: unknown error: Chrome failed to start: exited normally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location C:\Program Files\Google\Chrome\Application\chrome.exe is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x004CDCE3+50899]
	(No symbol) [0x0045E111]
	(No symbol) [0x00365588]
	(No symbol) [0x0038281C]
	(No symbol) [0x0037F479]
	(No symbol) [0x003B1FFE]
	(No symbol) [0x003B1CEC]
	(No symbol) [0x003AB6F6]
	(No symbol) [0x00387708]
	(No symbol) [0x0038886D]
	GetHandleVerifier [0x00733EAE+2566302]
	GetHandleVerifier [0x007692B1+2784417]
	GetHandleVerifier [0x0076327C+2759788]
	GetHandleVerifier [0x00565740+672048]
	(No symbol) [0x00468872]
	(No symbol) [0x004641C8]
	(No symbol) [0x004642AB]
	(No symbol) [0x004571B7]
	BaseThreadInitThunk [0x773200C9+25]
	RtlGetAppContainerNamedObjectPath [0x77567B4E+286]
	RtlGetAppContainerNamedObjectPath [0